# 🚀 JetX Model Eğitimi - Google Colab

**Bu notebook ile JetX tahmin modellerini Google Colab'da eğitebilirsiniz.**

## 🎯 İki Model Seçeneği:

### 1️⃣ Neural Network (Deep Learning) ~2-3 saat

- ✅ **En Yüksek Performans**: %75-85 doğruluk hedefi
- ✅ **Transformer**: 4-layer attention mechanism ile derin analiz
- ✅ **1000 Pencere**: Çok uzun vadeli pattern analizi (5 girdi katmanı)
- ✅ **3 Aşamalı Progressive Training**
- ✅ **Çift Sanal Kasa**: 2 farklı strateji test edilir
- ✅ **YENİ: Her Epoch Sanal Kasa Gösterimi** 💰
- ⚠️ **Uzun Eğitim**: 2-3 saat gerektirir
- 📊 **Model Boyutu**: ~15-18M parametre

### 2️⃣ CatBoost (Gradient Boosting) ~30-60 dakika

- ⚡ **Çok Hızlı**: 3-4x daha hızlı eğitim
- ✅ **Native Class Weights**: Daha iyi class imbalance yönetimi
- ✅ **Feature Engineering**: Zengin özellik analizi
- ✅ **Feature Importance**: Hangi özelliklerin önemli olduğunu gösterir
- ✅ **Çift Sanal Kasa**: 2 farklı strateji test edilir
- ✅ **YENİ: Her 10 Iteration Sanal Kasa Gösterimi** 💰
- ✅ **Daha Az Bellek**: Hafif ve verimli
- 📊 **Performans**: %70-80 doğruluk hedefi

## 🎲 Hangisini Seçmeli?

- **En yüksek performans istiyorsanız**: Neural Network (Transformer ile)
- **Hızlı bir model istiyorsanız**: CatBoost
- **İkisini de deneyin**: Karşılaştırın ve en iyi sonucu veren modeli kullanın!

## 💰 Çift Sanal Kasa Sistemi

Her iki model de 2 farklı strateji ile test edilir:

- **Kasa 1**: 1.5x eşik sistemi (klasik strateji)
- **Kasa 2**: %80 çıkış sistemi (2.0x+ tahminlerde dinamik çıkış)

En karlı strateji belirlenir ve karşılaştırma raporu sunulur.

## 🆕 YENİ: Canlı Sanal Kasa Takibi!

Artık eğitim sırasında **her epoch/iteration'da** sanal kasa simülasyonu göreceksiniz:

**Neural Network için (Her Epoch):**
```
================================================================================
💰 AŞAMA 1 - Epoch 1 - SANAL KASA SİMÜLASYONU
================================================================================
   🎲 Oyun: 1,234 el (850 kazanç, 384 kayıp)
   📊 Kazanma Oranı: 68.9%
   💰 Başlangıç: 1,000 TL → Final: 1,025 TL
   📈 Net: +25 TL | ROI: +2.50% ✅
   ✅ Kazanma oranı başabaş noktasının ÜSTÜNDE (%66.7)
   🏆 En İyi: Epoch 1 (ROI: +2.50%)
================================================================================
```

**CatBoost için (Her 10 Iteration):**
```
================================================================================
💰 CATBOOST REGRESSOR - Iteration 10 - SANAL KASA
================================================================================
   🎲 Oyun: 1,456 el (982 kazanç, 474 kayıp)
   📊 Kazanma Oranı: 67.4%
   💰 Başlangıç: 1,000 TL → Final: 1,010 TL
   📈 Net: +10 TL | ROI: +1.00% ✅
   🏆 En İyi: Iteration 10 (ROI: +1.00%)
================================================================================
```

Bu sayede modelin **gerçekte kar edip etmediğini** canlı olarak izleyebilirsiniz! 🎯

---
## 📦 Adım 1: Hazırlık (2-3 dakika)

**Önce bu hücreyi çalıştırın** - Her iki model için de gerekli

In [ ]:
#@title ▶️ Hazırlık - Buraya tıklayın { display-mode: "form" }

print("📦 Kütüphaneler yükleniyor...")
!pip install -q tensorflow scikit-learn catboost pandas numpy scipy joblib matplotlib seaborn tqdm PyWavelets nolds

print("\n📥 Proje GitHub'dan indiriliyor...")
import os
if os.path.exists('jetxpredictor'):
    !rm -rf jetxpredictor
!git clone https://github.com/onndd/jetxpredictor.git
%cd jetxpredictor

print("\n✅ Hazırlık tamamlandı!")
print("\n➡️ Şimdi aşağıdaki seçeneklerden BİRİNİ seçin:")
print("   • Neural Network için Adım 2A")
print("   • CatBoost için Adım 2B")

---

# 🧠 SEÇENEK A: Neural Network Eğitimi

## 🎯 3 Aşamalı Progressive Training (TRANSFORMER + 1000 PENCERE)

### AŞAMA 1: Foundation Training (100 epoch)

- Model hem değer tahmin etmeyi HEM DE 1.5 eşiğini birlikte öğrenir
- **Transformer**: 4-layer, 8-head attention mechanism
- **1000 Pencere**: X_1000 + nb_xl bloğu ile çok derin analiz
- Class Weight: 1.2x (çok yumuşak başlangıç - lazy learning'i önler)
- Patience: 12

### AŞAMA 2: Threshold Fine-Tuning (80 epoch)

- 1.5 altı/üstü ayrımını keskinleştirir
- Class Weight: 1.5x (orta seviye)
- Patience: 10

### AŞAMA 3: Full Model Fine-Tuning (80 epoch)

- Tüm output'ları birlikte optimize eder
- Class Weight: 2.0x (dengeli final)
- Patience: 8

**⏱️ Toplam Süre**: ~2-3 saat

**🎯 Hedefler**: 1.5 Altı %75-85+, 1.5 Üstü %75-85+, Para Kaybı <%20

**✨ Yenilikler**: Transformer encoder + Çift sanal kasa + **HER EPOCH SANAL KASA GÖSTERİMİ** + ZIP paketi

In [ ]:
#@title ▶️ Adım 2A: Neural Network Eğitimi BAŞLAT { display-mode: "form" }

print("🧠 NEURAL NETWORK - Progressive Training (Transformer) Başlatılıyor...")
print("⏱️ Tahmini süre: 2-3 saat (GPU ile)")
print("📊 Model: 5 girdi (features + seq_50/200/500/1000) + Transformer")
print("💰 YENİ: Her epoch sonunda sanal kasa simülasyonu!")
print("="*70)

!python notebooks/jetx_PROGRESSIVE_TRAINING.py

print("\n" + "="*70)
print("✅ Neural Network eğitimi tamamlandı!")
print("\n➡️ Şimdi Adım 3A'ya geçin (Neural Network modelini indirin)")

In [ ]:
#@title ▶️ Adım 3A: Neural Network Modelini İndir { display-mode: "form" }

from google.colab import files
import os

print("💾 Neural Network dosyaları indiriliyor...\n")

# ZIP dosyası mevcut mu kontrol et
zip_file = 'jetx_models_progressive_v2.0.zip'
if os.path.exists(zip_file):
    print(f"📥 {zip_file} indiriliyor...")
    files.download(zip_file)
    print(f"✅ {zip_file} indirildi")
    print("\n✅ Tüm Neural Network dosyaları tek ZIP'te indirildi!")
    print("\n📁 ZIP İÇERİĞİ:")
    print("   • jetx_progressive_transformer.h5 (Model - Transformer ile)")
    print("   • scaler_progressive_transformer.pkl (Scaler)")
    print("   • model_info.json (Model bilgileri + Çift kasa sonuçları)")
    print("\n📌 NASIL KULLANILIR:")
    print("   1. ZIP'i açın")
    print("   2. Tüm dosyaları lokal projenizin models/ klasörüne kopyalayın")
    print("   3. Streamlit uygulamasını yeniden başlatın")
else:
    # Fallback: Tek tek dosya indirme
    print("⚠️ ZIP dosyası bulunamadı, tek tek indiriliyor...\n")
    model_files = [
        'models/jetx_progressive_transformer.h5',
        'models/scaler_progressive_transformer.pkl',
        'models/model_info.json'
    ]
    
    for file in model_files:
        if os.path.exists(file):
            print(f"📥 {file} indiriliyor...")
            files.download(file)
            print(f"✅ {file} indirildi")
        else:
            print(f"⚠️ {file} bulunamadı!")
    
    print("\n✅ Tüm Neural Network dosyaları indirildi!")
    print("\n📁 Bu dosyaları lokal projenizin 'models/' klasörüne kopyalayın")

print("\n🎉 Kullanıma hazır!")

---

# ⚡ SEÇENEK B: CatBoost Eğitimi

## 🤖 Hızlı ve Verimli Model

### CatBoostRegressor (Değer Tahmini)

- iterations: 1500
- depth: 10
- learning_rate: 0.03
- loss_function: MAE
- task_type: GPU (varsa)

### CatBoostClassifier (1.5 Eşik Tahmini)

- iterations: 1500
- depth: 9
- learning_rate: 0.03
- class_weights: **Native destek** (Balanced)
- task_type: GPU (varsa)

**⏱️ Toplam Süre**: ~30-60 dakika

**🎯 Hedefler**: 1.5 Altı %70-80+, 1.5 Üstü %70-80+

**✨ Bonus**: Feature importance analizi + Çift sanal kasa + **HER 10 ITERATION SANAL KASA** + ZIP paketi!

In [ ]:
#@title ▶️ Adım 2B: CatBoost Eğitimi BAŞLAT { display-mode: "form" }

print("⚡ CATBOOST - Hızlı Model Eğitimi Başlatılıyor...")
print("⏱️ Tahmini süre: 30-60 dakika")
print("📊 Model: Feature Engineering bazlı + Native class weights")
print("💰 YENİ: Her 10 iteration'da sanal kasa simülasyonu!")
print("="*70)

!python notebooks/jetx_CATBOOST_TRAINING.py

print("\n" + "="*70)
print("✅ CatBoost eğitimi tamamlandı!")
print("\n➡️ Şimdi Adım 3B'ye geçin (CatBoost modelini indirin)")

In [ ]:
#@title ▶️ Adım 3B: CatBoost Modelini İndir { display-mode: "form" }

from google.colab import files
import os

print("💾 CatBoost dosyaları indiriliyor...\n")

# ZIP dosyası mevcut mu kontrol et
zip_file = 'jetx_models_catboost_v2.0.zip'
if os.path.exists(zip_file):
    print(f"📥 {zip_file} indiriliyor...")
    files.download(zip_file)
    print(f"✅ {zip_file} indirildi")
    print("\n✅ Tüm CatBoost dosyaları tek ZIP'te indirildi!")
    print("\n📁 ZIP İÇERİĞİ:")
    print("   • catboost_regressor.cbm (CatBoost Regressor)")
    print("   • catboost_classifier.cbm (CatBoost Classifier)")
    print("   • catboost_scaler.pkl (Scaler)")
    print("   • catboost_model_info.json (Model bilgileri + Çift kasa sonuçları)")
    print("\n📌 NASIL KULLANILIR:")
    print("   1. ZIP'i açın")
    print("   2. Tüm dosyaları lokal projenizin models/ klasörüne kopyalayın")
    print("   3. Predictor'ı model_type='catboost' ile başlatın")
    print("      predictor = JetXPredictor(model_type='catboost')")
else:
    # Fallback: models klasörüne gir ve tek tek indir
    print("⚠️ ZIP dosyası bulunamadı, tek tek indiriliyor...\n")
    os.chdir('models')
    
    model_files = [
        'catboost_regressor.cbm',
        'catboost_classifier.cbm',
        'catboost_scaler.pkl',
        'catboost_model_info.json'
    ]
    
    for file in model_files:
        if os.path.exists(file):
            print(f"📥 {file} indiriliyor...")
            files.download(file)
            print(f"✅ {file} indirildi")
        else:
            print(f"⚠️ {file} bulunamadı!")
    
    print("\n✅ Tüm CatBoost dosyaları indirildi!")
    print("\n📁 Bu dosyaları lokal projenizin 'models/' klasörüne kopyalayın")
    print("\n🔧 Kullanım: Predictor'ı model_type='catboost' ile başlatın")
    print("   predictor = JetXPredictor(model_type='catboost')")

print("\n🎉 Kullanıma hazır!")

---

## 📊 Metrik Açıklamaları

Eğitim sırasında göreceğiniz metriklerin anlamları:

### 🔴 1.5 ALTI DOĞRULUĞU

- Model, **gerçekten 1.5 altında olan** değerleri ne kadar doğru tahmin ediyor?
- **Hedef**: Neural Network %75-85+, CatBoost %70-80+
- **Neden önemli**: 1.5 altını bulamazsak fırsat kaçırırız

### 🟢 1.5 ÜSTÜ DOĞRULUĞU

- Model, **gerçekten 1.5 üstünde olan** değerleri ne kadar doğru tahmin ediyor?
- **Hedef**: Neural Network %75-85+, CatBoost %70-80+
- **Neden önemli**: Doğru tahmin edersek gereksiz bahisten kaçınırız

### 💰 PARA KAYBI RİSKİ

- Model **1.5 altı olduğunda** yanlışlıkla "1.5 üstü" deme oranı
- **Hedef**: <%20-25
- **Neden önemli**: Bu durumda bahis yapar ve PARA KAYBEDERİZ!

### 💰 SANAL KASA SİMÜLASYONU (YENİ!)

Her epoch/iteration'da şunu göreceksiniz:

```
💰 Başlangıç: 1,000 TL → Final: 1,025 TL
📈 Net: +25 TL | ROI: +2.50% ✅
📊 Kazanma Oranı: 68.9% (Başabaş: %66.7)
```

**Bu ne anlama geliyor?**
- **ROI (+)** = Model kar ediyor! ✅
- **ROI (-)** = Model zarar ediyor ❌
- **Kazanma > %66.7** = Başabaş noktası üstünde (Hedef!) ✅
- **Kazanma < %66.7** = Başabaş noktası altında (Risk!) ⚠️

**Başabaş Noktası:**
- 2 kazanç (2×5 TL) = 1 kayıp (10 TL) dengelemeli
- Minimum gerekli: %66.7 kazanma oranı

### 💰 ÇİFT SANAL KASA SİSTEMİ

Her iki model de 2 farklı strateji ile test edilir:

**Kasa 1: 1.5x Eşik Sistemi**
- Model 1.5x üstü tahmin ederse → 1.5x'te çıkış
- Klasik, güvenli strateji
- Hedef ROI: +%5-15

**Kasa 2: %80 Çıkış Sistemi**
- Model 2.0x+ tahmin ederse → Tahmin × 0.80'de çıkış
- Dinamik, agresif strateji
- Hedef ROI: +%10-25 (potansiyel daha yüksek)

**Karşılaştırma Raporu** otomatik olarak gösterilir ve en karlı strateji belirlenir.

---

## ⚠️ Önemli Notlar

- 🚨 Bu modeller %100 doğru değildir
- 💰 Yine de para kaybedebilirsiniz
- 🎯 1.5x kritik eşiktir
- 🛡️ Rolling mod en güvenlidir (%80+ güven)
- ⚡ Aggressive mod çok risklidir
- **İki modeli de deneyin** ve karşılaştırın!
- 🆕 **Transformer**: Neural Network'te derin zaman serisi analizi
- 🆕 **CatBoost**: XGBoost'tan daha iyi class imbalance yönetimi
- 🆕 **Çift kasa sistemi** ile risk yönetimi daha iyi
- 🆕 **Her epoch/iteration sanal kasa** ile canlı performans takibi!

**Sorumlu oynayın!** 🎲